In [33]:
# loading required libraries
from asyncio import as_completed
import torch 
import torchvision 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
from torchsummary import summary
from PIL import Image
from torch import optim
import cv2
import glob, numpy as np, pandas as pd
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset
import torch.utils.data as data
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
IMAGE_SIZE = 512

In [35]:
    train_transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.CenterCrop(size=224),
        transforms.GaussianBlur(kernel_size=(5,9), sigma=(0.1,5)),
        transforms.RandomAdjustSharpness(sharpness_factor=1, p=0.5),
        transforms.ToTensor(),
         transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])    
    ])
    #validation Transform
    valid_transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
         transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    #test Transform
    test_transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
         transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

## Data Loading

In [36]:
# Load the data
train_dir ="kaggle/train"
valid_dir = "kaggle/test"
bs = 64
num_classes = 5
# Load the Data from folders
data = {
    'train': datasets.ImageFolder(root = train_dir,
    transform =train_transform),
    'valid': datasets.ImageFolder(root = valid_dir,
    transform = valid_transform),
    'test': datasets.ImageFolder(root = valid_dir,
    transform = test_transform)
}

#Size of the data to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size  = len(data['test'])

#Creating iterators for the DataLoader using DataLoader module
train_data = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data = DataLoader(data['valid'],batch_size=bs, shuffle=True)
test_data = DataLoader(data['test'],batch_size=bs, shuffle=True)
print(train_data_size,valid_data_size,test_data_size)

24168 6056 6056


In [37]:
#Loading the pretrained model
model = models.resnet50(pretrained = True)

c:\Users\mkhan\Desktop\home\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\mkhan\Desktop\home\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [38]:
#Freeze the model
for param in model.parameters():
    param.requires_grad=False

In [39]:
#change the final layer of the model for transfer learning
fc_inputs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(fc_inputs,256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256,5),
    nn.LogSoftmax(dim=1) # for using NLLLoss()
)

In [40]:
model = model.to(device)

In [41]:
# Define the optimizer and Loss function
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

## Training

In [42]:
import time
epochs = 20
for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
    # set to training mode
    model.train()
    # loss and accuracy with in epochs
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    for i, (inputs, labels) in enumerate(train_data):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # clean the existing gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(inputs)
        #cumpute loss
        loss = loss_fn(outputs,labels)
        #back propagate the gradients
        loss.backward()
        #update the parameters
        optimizer.step()
        #compute the total loss for the batch
        train_loss += loss.item() * inputs.size(0)
        #compute the accuracy
        ret, predictions = torch.max(outputs.data,1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
        # convert the correct counts to float and then compute the mean
        acc= torch.mean(correct_counts.type(torch.FloatTensor))
        #compute the whole accuracy in the batch
        train_acc += acc.item() * inputs.size(0)
        print("Batch number : {:03d}, Training: Loss:  {:.4f}, Accuracy: {:.4f}".format(i, loss.item(),acc.item()))


Epoch: 1/20
Batch number : 000, Training: Loss:  1.7010, Accuracy: 0.0625
Batch number : 001, Training: Loss:  0.5532, Accuracy: 0.8594
Batch number : 002, Training: Loss:  0.5230, Accuracy: 0.8906
Batch number : 003, Training: Loss:  0.6184, Accuracy: 0.8594
Batch number : 004, Training: Loss:  0.5642, Accuracy: 0.8750
Batch number : 005, Training: Loss:  0.5167, Accuracy: 0.8906
Batch number : 006, Training: Loss:  0.9077, Accuracy: 0.8125
Batch number : 007, Training: Loss:  1.0081, Accuracy: 0.7812
Batch number : 008, Training: Loss:  0.2834, Accuracy: 0.9219
Batch number : 009, Training: Loss:  1.0343, Accuracy: 0.8125
Batch number : 010, Training: Loss:  0.8408, Accuracy: 0.7969
Batch number : 011, Training: Loss:  0.5132, Accuracy: 0.8906
Batch number : 012, Training: Loss:  0.7006, Accuracy: 0.7969
Batch number : 013, Training: Loss:  0.6286, Accuracy: 0.8281
Batch number : 014, Training: Loss:  0.6078, Accuracy: 0.8594
Batch number : 015, Training: Loss:  0.5616, Accuracy: 0.9

In [43]:
# Validation - No gradient tracking needed
history =[]
with torch.no_grad():
    # Set to evaluation mode
    model.eval()
    # Validation loop
    for j, (inputs, labels) in enumerate(valid_data):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Forward pass - compute outputs on input data using the model
        outputs = model(inputs)
        # Compute loss
        loss = loss_fn(outputs, labels)
        # Compute the total loss for the batch and add it to valid_loss
        valid_loss += loss.item() * inputs.size(0)
        # Calculate validation accuracy
        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))
        # Convert correct_counts to float and then compute the mean
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        # Compute total accuracy in the whole batch and add to valid_acc
        valid_acc += acc.item() * inputs.size(0)
        print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/float(train_data_size)
        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/float(valid_data_size)
        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
        epoch_end = time.time()
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, nttValidation : Loss : {:.4f},Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

Validation Batch number: 000, Validation: Loss: 0.5872, Accuracy: 0.8594
Epoch : 019, Training: Loss: 0.4694, Accuracy: 85.9442%, nttValidation : Loss : 0.0062,Accuracy: 0.9082%, Time: 291.7234s
Validation Batch number: 001, Validation: Loss: 0.6307, Accuracy: 0.7500
Epoch : 019, Training: Loss: 0.4694, Accuracy: 85.9442%, nttValidation : Loss : 0.0129,Accuracy: 1.7008%, Time: 292.6065s
Validation Batch number: 002, Validation: Loss: 0.5961, Accuracy: 0.8281
Epoch : 019, Training: Loss: 0.4694, Accuracy: 85.9442%, nttValidation : Loss : 0.0192,Accuracy: 2.5760%, Time: 293.4182s
Validation Batch number: 003, Validation: Loss: 0.2556, Accuracy: 0.9375
Epoch : 019, Training: Loss: 0.4694, Accuracy: 85.9442%, nttValidation : Loss : 0.0219,Accuracy: 3.5667%, Time: 294.2586s
Validation Batch number: 004, Validation: Loss: 0.3738, Accuracy: 0.9062
Epoch : 019, Training: Loss: 0.4694, Accuracy: 85.9442%, nttValidation : Loss : 0.0258,Accuracy: 4.5244%, Time: 295.1335s
Validation Batch number: 

## Inference

In [ ]:
def predict(model, test_image_name):
    transform = image_transforms['test']
    test_image = Image.open(test_image_name)
    plt.imshow(test_image)
    test_image_tensor = transform(test_image)
    if torch.cuda.is_available():
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
    else:
        test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_image_tensor)
        ps = torch.exp(out)
        topk, topclass = ps.topk(1, dim=1)
        print("Output class :  ", idx_to_class[topclass.cpu().numpy()[0][0]])